## Generalized Assignment Problem (GAP) Formulation

### Mathematical Formulation Description

Suppose there are $M = \{1, \cdots, m\}$ jobs and $N = \{1, \cdots, n\}$ agents. Each agent $j \in N$ has a maximum work capacity $b_j$. If a job $i \in M$ is assigned to an agent $j \in N$ , it generates a profit $c_{ij}$, but consumes $a_{ij}$ units of agent $j$’s work capacity $b_j$. Generalized Assignment Problem (GAP) is to find a maximum profit assignment of jobs to agents so that: \
    (1) Each job $i \in M$ is assigned to at most one agent $j \in N$. \
    (2) Total capacity of jobs assigned to an agent $j \in N$ does not exceed $b_j$.

The problem can be formualated as below:
\begin{align*}
    & \text{(GAP)} & \max_x & \sum_{i \in M} \sum_{j \in N} c_{ij} x_{ij} \\
    & & \text{s.t.:} & \sum_{j \in N} x_{ij} \leq 1, \ \ \forall i \in M, \\
    & & & \sum_{i \in M} a_{ij} x_{ij} \leq b_j, \ \ \forall j \in N, & \\
    & & & x_{ij} \in \{0, 1 \}, \ \ \forall i \in M, \forall j \in N, 
\end{align*}
where $x_{ij} = 1$ if job $i \in M$ is assigned to agent $j \in N$, and $x_{ij} = 0$ otherwise.

We implement the problem as per this formulation into a JuMP model as below:

In [1]:
# Import package
using JuMP, Cbc

### Define the corresponding JuMP model

In [58]:
function GAP_formulation(optimizer, a, b, c)
    """
    optimizer: solver optimizer
    a: Unit consumption of work capacity: assigning job i to agent j consumes a[i][j] work capacity
    b: Work capacity of agents: agent j has work capacity b[j] 
    c: Unit profit: assigning job i to agent j generates c[i][j] profit
    """
    model = Model(optimizer)

    # Dimension of variables x[i][j]
    ## Number of jobs nI and agents nJ
    nI, nJ = size(c)

    # Define sets
    ## Set of jobs (i ∈ M)
    M = 1:nI
    ## Set of agents (j ∈ N)
    N = 1:nJ

    # Define variable x with corresponding dimension
    @variable(model, x[M, N], Bin)

    # Define objective function
    @objective(model, Max, sum(c[i,j]*x[i,j] for i in M for j in N))

    # Define constraint: each job i ∈ M is assigned to at most one agent j ∈ N
    @constraint(model, unique_assignment[i in M], sum(x.data[i,j] for j in N) ≤ 1)

    # Define constraint: total capacity of jobs assigned to an agent j ∈ N does not exceed b[j]
    @constraint(model, agent_capacity[j in N], sum(a[i,j]*x[i,j] for i in M) ≤ b[j])

    return model
end

GAP_formulation (generic function with 1 method)

### Function to solve the model

In [49]:
function solve_model!(model; silent::Bool=false)
    if silent
        set_silent(model)
    end
    optimize!(model)

    status = termination_status(model)
    println(status)
end

solve_model! (generic function with 2 methods)

### Function to print solutions

In [35]:
function print_solution(model)
    if termination_status(model) == OPTIMAL
        println("Optimal solution found!")
        println("Optimal value: $(objective_value(model))")
        println("with x = $(value.(model[:x].data))")
    else
        println("Solving of the model terminated due to $(termination_status(model))")
    end
end

print_solution (generic function with 1 method)

### Implementing the JuMP model into the given instance
#### Data of the instance

In [46]:
# Unit profit
c_ins = [6 2;
         7 7;
         4 8;
         3 3]

# Unit consumption of work capacity
a_ins = [5 3;
         7 1;
         4 6;
         2 4]

# Work capacity of agents
b_ins = [7; 6]
;

#### Implement the model to the instance and solve

In [60]:
model_gap = GAP_formulation(Cbc.Optimizer, a_ins, b_ins, c_ins)
solve_model!(model_gap, silent=true)

OPTIMAL


### Print solution

In [61]:
print_solution(model_gap)

Optimal solution found!
Optimal value: 17.0
with x = [1.0 0.0; 0.0 0.0; 0.0 1.0; 1.0 0.0]
